In [3]:
# Learning on how to prepare PascalVOC to Yolov5 format
# Import Required libaries

import numpy as np # manage on linear algebra
import pandas as pd # data processing, CSV file I/O

from pathlib import Path #Represent file system path

#Manage DOM(The Document Object Model) -> API XML Document
from xml.dom.minidom import parse 

from shutil import copyfile #High-Level Operation on Files

import os #Provide function to interact with OS

#Test on os.walk function
# for dirname, _, filenames in os.walk('/home/jarupong/Desktop/helmetDetection'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

In [6]:
#Prepare directory for Yolov5 format
!mkdir -p Dataset/labels
!mkdir -p Dataset/images

In [4]:
#Config on class of object in model
classes = ['helmet','head','person']

In [5]:
#This function will transform PasvalVOC to Yolov5 format
#by parse input and calculate to BB algorithm.
def convertFromXML(parsed_img_size, box):
    x1=int(box[0])
    y1=int(box[1])
    x2=int(box[2])
    y2=int(box[3])

    dw = np.float32(1. / int(parsed_img_size[0]) )
    dh = np.float32(1. / int(parsed_img_size[1]) )

    w = x2 - x1
    h = y2 -y1
    x = x1 + (w/2)
    y = y1 + (h/2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h *dh

    return [x, y, w, h]

In [6]:
#This function will create new annotation file in Yolov5 format
def save_txt_file(img_jpg_file_name, parsed_img_size, label_box):
    save_file_name = 'result/working/Dataset/labels/' + img_jpg_file_name +'.txt'
    print(save_file_name)

    with open(save_file_name, 'a+') as file_path:
        for box in label_box:

            class_num = classes.index(box[0])

            yolov5_box = convertFromXML(parsed_img_size, box[1:])

            file_path.write(f"{class_num} {yolov5_box[0]} {yolov5_box[1]} {yolov5_box[2]} {yolov5_box[3]}\n")

        file_path.flush()
        file_path.close()

In [7]:
#This function will access and parsing on original PasvalVOC format 
def get_xml_data(file_path, img_xml_file):
    img_path = file_path + '/' + img_xml_file + '.xml'
    (print(img_path))

    dom = parse(img_path)
    root = dom.documentElement
    img_name = root.getElementByTagName("filename")[0].childNodes[0].data
    img_size = root.getElementByTagName("size")[0]
    objects = root.getElementByTagName("object")
    img_w = img_size.getElementByTagName("width").childNodes[0].data
    img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data
    img_c = img_size.getElementsByTagName("depth")[0].childNodes[0].data

    label_box = []
    for box in objects:
        class_name = box.getElementsByTagName("name")[0].childNodes[0].data
        x1 = box.getElementsByTagName("xmin")[0].childNodes[0].data
        y1 = box.getElementsByTagName("ymin")[0].childNodes[0].data
        x2 = box.getElementsByTagName("xmax")[0].childNodes[0].data
        y2 = box.getElementsByTagName("ymax")[0].childNodes[0].data

        img_jpg_file_name = img_xml_file + '.jpg'
        label_box.append([class_name, x1, y1, x2, y2])

    save_txt_file(img_xml_file, [img_w, img_h], label_box)

In [11]:
!ls

Dataset  PascalVOC  README.md  testPython.ipynb  trainModel.ipynb
